In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re
import csv
import pandas as pd
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List

In [2]:
# 対象競馬場とレース
PLACE_CODES = [
    "35", "45", "55", "65", "46", "54", "44",
    "47", "50", "30", "36", "43", "48", "42"
]
# レース番号
RACE_NUMBERS = [f"{i:02d}" for i in range(1, 13)]
#設定値
LOGIN_URL = "https://regist.netkeiba.com/account/?pid=login"
LOGIN_ID = "keibaaitool@gmail.com"
LOGIN_PASSWORD = "keibaai"
RACE_LIST_URL = "https://nar.netkeiba.com/top/race_list.html?kaisai_date="

In [9]:
# 1. 馬データ
@dataclass
class Horse:
    """出走馬のデータを保持するデータクラス"""
    horse_id: str
    horse_name: str
    jockey_id: str
    jockey_name: str
    popularity: int
    odds: float
    finish_time: str
    time_index: int
    weight: float
    horse_number: int
    frame_number: int
    style_name_pred: int
    finish_rank: int
    note: str
    position: str
    impost: int
    last_3_furlongs: float

# 2. レースデータ
@dataclass
class Race:
    """レースの基本情報と出走馬リストを保持するデータクラス"""
    race_id: str
    race_name: str
    race_date: str
    race_time: str
    num_horses: str
    race_number: int
    weather_name: str
    track_condition_name: str
    racecourse: str
    ground: str
    distance: int
    direction:  str
    note: str
    horses: List[Horse] = field(default_factory=list)

In [33]:
def is_xpath_present(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath)) > 0

def login_netkeiba(driver):
    # ログイン
    driver.get(LOGIN_URL)
    driver.find_element(By.NAME, "login_id").send_keys(LOGIN_ID)
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys(LOGIN_PASSWORD)
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

def scrape_horse_data(driver, num_horses):

    all_horses_data = []
    # 馬番・馬名取得
    # 馬番・馬名・馬名リンク取得
    for i in range(1, num_horses + 1):
        try:
            # 馬番
            horse_number_xpath = f'//*[@id="tr_{i}"]/td[2]'
            horse_number = driver.find_element(By.XPATH, horse_number_xpath).text.strip()
            # 馬名と馬ID
            horse_link_xpath_1 = f'/html/body/div[1]/div[3]/div[3]/table/tbody/tr[{i}]/td[4]/div/div/span[2]/a'
            horse_link_xpath_2 = f'/html/body/div[1]/div[3]/div[3]/table/tbody/tr[{i}]/td[4]/div/div/span/a'
            if is_xpath_present(driver, horse_link_xpath_1):
                horse_link_xpath = horse_link_xpath_1
            elif is_xpath_present(driver, horse_link_xpath_2):
                horse_link_xpath = horse_link_xpath_2
            horse_link_elem = driver.find_element(By.XPATH, horse_link_xpath)
            horse_name = horse_link_elem.text.strip()
            horse_href = horse_link_elem.get_attribute("href")
            horse_id = horse_href.rstrip("/").split("/")[-1]
            # 騎手名と騎手ID
            jockey_link_xpath = f'//*[@id="tr_{i}"]/td[7]/span/a'
            jockey_link_elem = driver.find_element(By.XPATH, jockey_link_xpath)
            jockey_name = jockey_link_elem.text.strip()
            jockey_href = jockey_link_elem.get_attribute("href")
            jockey_id = jockey_href.rstrip("/").split("/")[-1]

            horse_data = Horse(
                horse_id = horse_id,
                horse_name = horse_name,
                jockey_id = jockey_id,
                jockey_name = jockey_name,
                # popularity = popularity
                # odds = odds
                # finish_time = finish_time
                # time_index = time_index
                # weight = weight
                horse_number = horse_number
                # frame_number = frame_number
                # style_name_pred = style_name_pred
                # finish_rank = finish_rank
                # note = note
                # position = position
                # impost = impost
                # last_3_furlongs = last_3_furlongs
            )
            print(f"  馬番{i}: {horse_data.horse_number} | 馬名: {horse_data.horse_name} | 馬ID: {horse_data.horse_id} | 騎手名: {horse_data.jockey_name} | 騎手ID: {horse_data.jockey_id}")
            all_horses_data.append(horse_data)
        except Exception as e:
            print(f"  馬番{i}: データ取得失敗 ({e})")
    # 戻り値
    return all_horses_data

def scrape_race_data(driver, race_id, date):

    try:
        # 競馬場
        racecourse_elem = driver.find_element(By.CSS_SELECTOR,
            "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(2)")
        racecourse = racecourse_elem.text
        # R
        race_number_elem = driver.find_element(By.CSS_SELECTOR,
            "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item01 > div")
        race_number = race_number_elem.text
        # 頭数
        num_horses_elem = driver.find_element(By.CSS_SELECTOR,
            "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(5)")
        num_horses_text = num_horses_elem.text
        num_horses = int(num_horses_text.replace("頭", "").strip())
        # レース情報（まとめられている）
        race_info_xpath = '//*[@id="Netkeiba_Race_Nar_Shutuba"]/div[1]/div[2]/div/div[1]/div[3]/div[2]/div[2]'
        race_info_elem = driver.find_element(By.XPATH, race_info_xpath)
        race_info_text = race_info_elem.text.strip()
        # "/"で分割
        parts = [p.strip() for p in race_info_text.split("/")]            
        # 発送時刻を抽出
        race_time = parts[0].replace("発走", "").strip()
        ground_info = parts[1]
        # グラウンドを抽出
        ground_match = re.search(r"(芝|ダ)", ground_info)
        ground = ground_match.group(1) if ground_match else "不明"
        # 距離を抽出
        distance_match = re.search(r"(\d+)m", ground_info)
        distance = int(distance_match.group(1)) if distance_match else -1
        # 向きを抽出
        direction_match = re.search(r"\((左|右)\)", ground_info)
        direction = direction_match.group(1) if direction_match else "不明"
        # 出力確認
        print(f"\n📄 Race ID: {race_id} | 競馬場: {racecourse} | R: {race_number} | 頭数: {num_horses} | 🕓 発走時刻: {race_time} | 🏇 グラウンド: {ground} | 📏 距離: {distance}m | ↩ 回り: {direction}")
        # 馬のデータを抽出        
        horse_list = scrape_horse_data(driver, num_horses)

        race_data = Race(
            race_id = race_id,
            # race_name
            race_date = date,
            race_time = race_time,
            num_horses = num_horses,
            race_number = race_number,
            # weather_name
            # track_condition_name,
            racecourse = racecourse,
            ground = ground,
            distance = distance,
            # direction
            # note
            horses = horse_list
        ) 

    except Exception as e:
        print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
        return None
    # 戻り値
    return race_data

def scrape_one_day_data(driver, date):
    """
    競馬場とRでループ
    """
    all_races_data = []
    # 競馬場のループ
    for place_code in PLACE_CODES:
        for race_num in RACE_NUMBERS:
            yyyy = date.strftime("%Y")
            mmdd = date.strftime("%m%d")            
            race_id = f"{yyyy}{place_code}{mmdd}{race_num}"
            race_url = f"https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}"
            driver.get(race_url)
            time.sleep(1)
            race_data = scrape_race_data(driver, race_id, date)
            all_races_data.append(race_data)
    # 戻り値
    return all_races_data

def main_scraper(driver, date_list):
    """
    年月日でループ
    """
    all_days_data = []
    # 年月日のループ
    for date in date_list:
        # 年月日のURLでアクセス
        date_str = date.strftime("%Y%m%d")
        race_list_url = RACE_LIST_URL + date_str
        driver.get(race_list_url)
        one_day_data = scrape_one_day_data(driver, date)
        all_days_data.append(one_day_data)
    # 戻り値
    return all_days_data


In [35]:
# スクレイピング準備
service = Service('C:\\Users\\ryo\\AppData\\Local\\SeleniumBasic\\chromedriver.exe')
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
# ログイン
login_netkeiba(driver)
# 調子偏差値取得
start_date_str = "20250101"
end_date_str = "20250101"
start_date = datetime.strptime(start_date_str, "%Y%m%d")
end_date = datetime.strptime(end_date_str, "%Y%m%d")
# 開始日～終了日
date_list = pd.date_range(
    start=start_date_str,
    end=end_date_str,
    freq='D'
)
'''
# 3. for ループでインデックスを使わずに処理
# date_index から日付オブジェクトを一つずつ取り出す
for current_date in date_list:
    # YYYYMMDD形式の文字列に変換
    date_str = current_date.strftime("%Y%m%d")
    print(f"処理対象日: {date_str}")
    race_list_url = RACE_LIST_URL + date_str
    driver.get(race_list_url)
'''
final_result = main_scraper(driver, date_list)

# スクレイピング終了
driver.quit()

❌ Race ID: 202535010101 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010102 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010103 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010104 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010105 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010106 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010107 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010108 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010109 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010110 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010111 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202535010112 データ取得に失敗: invalid literal for int() with base 10: ''

📄 Race ID: 202545010101 | 競馬場: 川崎 | R: 1R | 頭数: 13 | 🕓 発走時刻: 11:20 | 🏇 グラウン

Exception ignored in: <function Service.__del__ at 0x000001F278FE62A0>
Traceback (most recent call last):
  File "C:\Users\ryo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 197, in __del__
    self.stop()
  File "C:\Users\ryo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 153, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\ryo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 138, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ryo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 127, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ryo\anaconda3\Lib\site-packages\selenium\webdriver\common\utils.py", line 122, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [27]:
# 地方競馬 https://nar.netkeiba.com/top/race_list.html?kaisai_date=YYYYMMDD
# 波乱度 https://nar.sp.netkeiba.com/race/compatibility.html
# 調子偏差値 https://race.sp.netkeiba.com/barometer/profile.html?kaisai_date=YYYYMMDD